In [5]:
%pwd

'c:\\Users\\PrinciaFernandes\\Mresult\\Phenomix\\notebooks'

In [6]:
import os
os.chdir("../")

In [7]:
%pwd

'c:\\Users\\PrinciaFernandes\\Mresult\\Phenomix'

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
from datetime import datetime


# service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service)

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

 # Initialize the driver with headless options



def phekb_scrapping(base_url):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(base_url)
    time.sleep(3)  
    all_links = []
    while True:
        rows = driver.find_elements(By.CSS_SELECTOR, "tbody.p-datatable-tbody tr.ng-star-inserted")

        for row in rows:
            try:
                title_element = row.find_element(By.CSS_SELECTOR, "td.ng-star-inserted a")
                title = title_element.text.strip()
                url = title_element.get_attribute("href")
                all_links.append((title, url))
            except Exception as e:
                print("Error extracting data:", e)
        
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "button.p-paginator-next")
            if "p-disabled" in next_button.get_attribute("class"):
                break
            next_button.click()
            time.sleep(3)
        except Exception as e:
            print("Pagination finished or error:", e)
            break
    phekb = process_all_links(all_links[:20],driver)
    driver.quit()
    return phekb

def extract_list(soup,selector):
    elements = soup.select(selector)
    return ', '.join([el.get_text(strip=True) for el in elements] if elements else ["NA"])

def process_all_links(all_links,driver):
    
    phenotypes_dictionary = []
    i = 0
    for title, url in all_links: 
        i += 1
        print(f"{i}: Title:",title) 
        driver.get(url)
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        phenotype_id = soup.select_one("div.field-name-field-phenotype-id div.field-items").get_text(strip=True) if soup.select_one("div.field-name-field-phenotype-id div.field-items") else "NA"
        name = soup.select_one("h1.title").get_text(strip=True) if soup.select_one("h1.title") else "NA"
        description = soup.select_one("div.field-name-body div.field-items").get_text(strip=True) if soup.select_one("div.field-name-body div.field-items") else "NA"
        status = soup.select_one("div.field-name-field-p-status div.field-items").get_text(strip=True) if soup.select_one("div.field-name-field-p-status div.field-items") else "NA"
        authors = extract_list(soup,"div.field-name-field-author div.field-items div.field-item")
        phenotype_attributes = extract_list(soup,"div.field-name-field-class div.field-items div.field-item")
        institutions = extract_list(soup,"div.field-name-field-institution div.field-items div.field-item")
        networks = extract_list(soup,"div.field-name-field-network-associations div.field-items div.field-item")
        age = extract_list(soup,"div.field-name-field-ages div.field-items div.field-item")
        races = extract_list(soup,"div.field-name-field-race div.field-items div.field-item")
        gender = extract_list(soup,"div.field-name-field-gender div.field-items div.field-item")
        type_of_phenotype = extract_list(soup,"div.field-name-field-pgx-type div.field-items")

        date_created = soup.select_one("div.field-name-field-created-at .field-items").get_text(strip=True) if soup.select_one("div.field-name-field-created-at .field-items") else "NA"
        date_new = datetime.strptime(date_created,"%A, %B %d, %Y")
        formatted_date = date_new.strftime("%Y-%m-%dT%H:%M:%S.000Z")
        
        files_section = soup.select_one("div.field-name-field-files div.field-items")
        files = {}
        if files_section:
            file_links = files_section.find_all("a")
            for file in file_links:
                files[file.get_text(strip=True)] = file.get("href", "NA")
        else:
            files = {"NA": "NA"}
        
        phenotype_data = {
            "Phenotype_id": int(phenotype_id),
            "Name": name,
            "Description": description,
            "Status": status,
            "Type_of_phenotype": type_of_phenotype ,
            "Phenotype_attributes": phenotype_attributes,
            "Authors": authors if authors != ["NA"] else ["NA"],
            "Files": files,
            "Institutions": institutions,
            "Ages": age ,
            "Races": races ,
            "Genders": gender ,
            "Networks": networks ,
            "Date_created": formatted_date
        }

        phenotypes_dictionary.append(phenotype_data)

    return phenotypes_dictionary


In [2]:
from src.utils import save_json
from src.config import RAW_DIR,PHEKB_URL

In [14]:

def main():
    base_url = "https://phekb.org/phenotypes"
    phekb = phekb_scrapping(base_url)
    # save_json(RAW_DIR,phekb,'PHEKB')
    return phekb

phekb = main()

1: Title: Abdominal Aortic Aneurysm ( AAA )
2: Title: ACE Inhibitor (ACE-I) induced cough
3: Title: Acute kidney injury (AKI)
4: Title: ADHD phenotype algorithm
5: Title: Anxiety algorithm
6: Title: Appendicitis
7: Title: Asthma
8: Title: Asthma Response to Inhaled Steroids
9: Title: Atopic Dermatitis Algorithm
10: Title: Atrial Fibrillation - Demonstration Project
11: Title: Autism
12: Title: Autoimmune Disease Phenotype
13: Title: Body Mass Index (BMI)
14: Title: bone scan utilization
15: Title: Breast Cancer
16: Title: CAAD (Carotid Artery Atherosclerosis Disease)
17: Title: caMRSA
18: Title: Cardiac Conduction (QRS)
19: Title: Cardiorespiratory Fitness Algorithm (eMERGE Mayo Network Phenotype)
20: Title: Cataracts


In [10]:
phekb

[{'Phenotype_id': 97,
  'Name': 'Abdominal Aortic Aneurysm ( AAA )',
  'Description': 'The algorithm uses Structured Query Language to identify AAA cases, controls, and excludes from the Electronic Medical Record. AAA cases were defined as meeting at least one of three criteria: had a AAA repair procedure (Case Type 1), had at least one vascular clinic encounter with a diagnosis of ruptured AAA (Case Type 2), or had at least two vascular clinic encounters with a diagnosis of unruptured AAA (Case Type 3). AAA controls must be neither excludes nor cases, had an encounter within the past 5 years, and never had a diagnosis of ICD9 code 441.3, 441.4, or 441.9 from any department. Patients were excluded if any exclusion diagnosis existed in the medical record, were younger than 40 or older than 89, were neither a Case Type 1 nor Case Type 2 and haven’t had an encounter within the past 5 years, or were not a case and had an ICD9 code diagnosis of 441.* at some point in the medical history.',


In [ ]:


def get_detail(phekb):
    phekb_detail = phekb
    i = 0
    for detail in phekb_detail:
        i += 1
        detail['Phenotype_id'] = int(detail['Phenotype_id'])
        detail['PID'] = f'PP{i:06d}'
    return phekb_detail

def get_concept(phekb,detail):
    phekb_concept = []
    i = 0
    for data in phekb:
        i += 1
        concept_json = {}
        concept_json['Files'] = data['Files']
        concept_json['Phenotype_attributes'] = data['Phenotype_attributes']
        concept_json['Phenotype_id'] = data['Phenotype_id']
        concept_json['Name'] = data['Name']
        concept_json['PIDs'] = [dict['PID'] for dict in detail if dict['Phenotype_id'] == data['Phenotype_id']]
        concept_json['CID'] = f'PC{i:06d}'
        phekb_concept.append(concept_json)
    return phekb_concept
    



In [ ]:
from src.utils import save_detail,save_concept
from src.config import PHEKB_DIR


In [17]:

def main():
    # dir = r'PheKB/'
    # with open(rf'{dir}\phekb_phenotypes.json','r') as file:
    #     data = file.read()
    #     phekb = json.loads(data)
    detail = get_detail(phekb)
    concept = get_concept(phekb,detail)
    save_detail(PHEKB_DIR,detail,'PHEKB')
    save_concept(PHEKB_DIR,concept,'PHEKB')
    return detail,concept

phekb_detail,phekb_concept = main()

